In [1]:
import cv2
import time
import face_recognition
import numpy as np
import os

In [2]:
# Database of known faces
known_face_encodings = []  # List of face encodings
known_face_metadata = []  # List of metadata corresponding to known faces
face_recognition_database_path = "Database"


# Function to add known faces
def load_known_faces():
    global known_face_encodings, known_face_metadata
    if not os.path.exists(face_recognition_database_path):
        print(f"[!] Directory {face_recognition_database_path} does not exist.")
        return

    for folder in os.listdir(face_recognition_database_path):
        path = os.path.join(face_recognition_database_path, folder)
        if not os.path.isdir(path):
            continue

        for filename in os.listdir(path):
            if filename.endswith('.jpg') or filename.endswith('.png'):
                image_path = os.path.join(path, filename)
                try:
                    face_image = face_recognition.load_image_file(image_path)
                    face_encodings = face_recognition.face_encodings(face_image)
                    if face_encodings:
                        face_encoding = face_encodings[0]
                        name = folder
                        metadata = {"name": name}
                        known_face_encodings.append(face_encoding)
                        known_face_metadata.append(metadata)
                        print(f"Loaded {name} from {image_path}")
                except Exception as e:
                    print(f"[!] Error processing {image_path}: {e}")

# Load known faces
load_known_faces()
print(known_face_metadata)
# print(known_face_encodings)

Loaded Jack from Database\Jack\1.jpg
Loaded Kareem from Database\Kareem\1.jpg
[{'name': 'Jack'}, {'name': 'Kareem'}]


In [7]:
def identify_person(frame: np.ndarray):
    rgb_frame = np.ascontiguousarray(frame[:, :, ::-1])
    face_locations = face_recognition.face_locations(rgb_frame)
    face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)
    
    closest_face_distance = float('inf')
    closest_face_name = "Unknown"
    closest_face_location = (0, 0, 0, 0)

    for face_encoding, (top, right, bottom, left) in zip(face_encodings, face_locations):
        
        matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
        if True in matches:
            first_match_index = matches.index(True)
            closest_face_name = known_face_metadata[first_match_index]["name"]
            closest_face_location = (top, right, bottom, left)
    
    
    return closest_face_name, closest_face_location

In [8]:
stream_src = 0
cap = cv2.VideoCapture(stream_src)

In [9]:
# Create a named window and set the size
cv2.namedWindow('Face Rec Inference', cv2.WINDOW_NORMAL)
cv2.resizeWindow('Face Rec Inference', 1024, 768)


In [10]:
while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to read frame or end of video")
        break
    
    # convert the frame to numpy array
    frame_t = np.array(frame)
    
    name, face_location = identify_person(frame_t)
    print(name)


    # display the close point
    # cv2.circle(frame, close_point, 5, (0, 255, 0), -1)
    # Display the results on the video frame
    if name == "Unknown":
        for (top, right, bottom, left), name in zip(face_recognition.face_locations(frame), [name]):
            # Draw a box around the face
            cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
            font = cv2.FONT_ITALIC
            cv2.putText(frame, name, (left + 6, top - 6), font, 0.5, (0, 255, 0), 1)
    else:
        top, right, bottom, left = face_location
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
        font = cv2.FONT_ITALIC
        cv2.putText(frame, name, (left + 6, top - 6), font, 0.5, (0, 255, 0), 1)
        
    cv2.imshow('Face Rec Inference', frame)

    # Wait for the correct amount of time to maintain the frame rate
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Kareem
Kareem
Kareem
Kareem
Kareem
Kareem
Kareem
Kareem
Kareem
Unknown
Kareem
Kareem
Kareem
Kareem
Kareem
Kareem
Kareem
Kareem
Kareem
Kareem
Kareem
Kareem
Unknown
Kareem
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown
Unknown


AttributeError: 'cv2.VideoCapture' object has no attribute 'stop'

: 